In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

Traceback (most recent call last):
  File "c:\Users\phamg\.vscode\extensions\ms-python.python-2024.20.0-win32-x64\python_files\python_server.py", line 130, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 3, in <module>
ModuleNotFoundError: No module named 'matplotlib'



In [ ]:
original_df = pd.read_csv('weather_data.csv', low_memory=False)

In [ ]:
original_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82536 entries, 0 to 82535
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   TIME UTC                      82536 non-null  object 
 1   Temp Definition °C            82287 non-null  float64
 2   Dew Point Definition °C       82288 non-null  float64
 3   Rel Hum Definition %          82288 non-null  float64
 4   Precip. Amount Definition mm  82288 non-null  float64
 5   Wind Dir Definition 10's deg  38548 non-null  float64
 6   Wind Spd Definition km/h      38548 non-null  float64
 7   Visibility Definition km      0 non-null      float64
 8   Stn Press Definition kPa      82288 non-null  float64
 9   Hmdx Definition               13528 non-null  float64
 10  Wind Chill Definition         0 non-null      float64
 11  Weather Definition            82288 non-null  object 
dtypes: float64(10), object(2)
memory usage: 7.6+ MB


In [ ]:
processed_df = pd.read_csv("preProcessedData.csv", low_memory=False)

processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3439 entries, 0 to 3438
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              3439 non-null   int64  
 1   Month             3439 non-null   int64  
 2   Day               3439 non-null   int64  
 3   Temp_mean         3439 non-null   float64
 4   Temp_max          3439 non-null   float64
 5   Temp_min          3439 non-null   float64
 6   DewPoint_mean     3439 non-null   float64
 7   DewPoint_max      3439 non-null   float64
 8   DewPoint_min      3439 non-null   float64
 9   RelHum_mean       3439 non-null   float64
 10  RelHum_max        3439 non-null   float64
 11  RelHum_min        3439 non-null   float64
 12  PrecipAmount_sum  3439 non-null   float64
 13  StnPress_mean     3439 non-null   float64
 14  StnPress_max      3439 non-null   float64
 15  StnPress_min      3439 non-null   float64
dtypes: float64(13), int64(3)
memory usage: 430

In [ ]:
processed_df.head()

   Year  Month  Day  ...  StnPress_mean  StnPress_max  StnPress_min
0  2015      1    1  ...         99.931        100.45         99.53
1  2015      1    2  ...        100.519        101.31         99.73
2  2015      1    3  ...        101.142        101.62         99.98
3  2015      1    4  ...         99.089         99.87         98.55
4  2015      1    5  ...        100.647        101.61         99.43

[5 rows x 16 columns]


In [ ]:
Q1 = processed_df[['Temp_mean', 'DewPoint_mean', 'RelHum_mean', 'PrecipAmount_sum', 'StnPress_mean']].quantile(0.25)

Q3 = processed_df[['Temp_mean', 'DewPoint_mean', 'RelHum_mean', 'PrecipAmount_sum', 'StnPress_mean']].quantile(0.75)

IQR = Q3 - Q1

print("Q1: ", Q1, "\n")
print("Q3: ", Q3, "\n")
print("IQR: ", IQR, "\n")

Q1:  Temp_mean            2.0375
DewPoint_mean       -3.7670
RelHum_mean         57.7920
PrecipAmount_sum     0.0000
StnPress_mean       99.7785
Name: 0.25, dtype: float64 

Q3:  Temp_mean            19.1120
DewPoint_mean        11.4165
RelHum_mean          73.4790
PrecipAmount_sum      1.3000
StnPress_mean       100.7145
Name: 0.75, dtype: float64 

IQR:  Temp_mean           17.0745
DewPoint_mean       15.1835
RelHum_mean         15.6870
PrecipAmount_sum     1.3000
StnPress_mean        0.9360
dtype: float64 



In [ ]:
processed_df[['Temp_mean', 'DewPoint_mean', \
              'RelHum_mean', 'PrecipAmount_sum', 'StnPress_mean']] \
            .boxplot(grid=False, vert=False, fontsize=15, figsize=(10, 5))

Axes(0.125,0.11;0.775x0.77)


In [ ]:
processed_df[['Temp_mean', 'DewPoint_mean', \
              'RelHum_mean', 'PrecipAmount_sum', 'StnPress_mean']] \
            .describe()

         Temp_mean  DewPoint_mean  RelHum_mean  PrecipAmount_sum  StnPress_mean
count  3439.000000    3439.000000  3439.000000       3439.000000    3439.000000
mean      9.911634       3.213395    65.379223          2.209218     100.242320
std      10.047182       9.594449    12.105542          5.869038       0.724695
min     -20.504000     -26.329000    24.458000          0.000000      97.090000
25%       2.037500      -3.767000    57.792000          0.000000      99.778500
50%       9.588000       2.921000    66.000000          0.000000     100.240000
75%      19.112000      11.416500    73.479000          1.300000     100.714500
max      28.742000      22.221000    95.292000         84.900000     102.680000


In [ ]:
processed_df[['StnPress_mean']].boxplot(grid=False, vert=False, fontsize=15, figsize=(10, 5))

Axes(0.125,0.11;0.775x0.77)


In [ ]:
processed_df[['PrecipAmount_sum']].boxplot(grid=False, vert=False, fontsize=15, figsize=(10, 5))

Axes(0.125,0.11;0.775x0.77)


### Fill humidex index column

- Setup humidex calculate function

$$H = T_{air} + \frac{5}{9} (6.11 exp(5417.7530(\frac{1}{273.15} - \frac{1}{273.15 + T_{dew}})) - 10)$$

  - **Where:**
    - **$H$:** denotes the Humidex
    - **$T_{air}$:** is the air temperature in °C
    - **$T_{dew}$:** is the dewpoint temperature in °C
    - **$exp$:** is the exponential function

  - _Reference: [Wikipedia](https://en.wikipedia.org/wiki/Humidex)_

In [ ]:
def hmdx_cal(temp, dew):
    try:
        temp = float(temp)
        dew = float(dew)
    except(ValueError, TypeError):
        return -1
    
    result = temp + (5/9) * (6.11 * math.exp(5417.7530 * ((1 / 273.15)-(1 / (273.15 + dew))))-10)

    return max(result, 0) # Because if humidex < 0, then it is not necessary

- Use `hmdx_cal` to calculate humidex

In [ ]:
processed_df['HMDX'] = processed_df.apply(lambda row: round(hmdx_cal(row['Temp_mean'], row['DewPoint_mean']), 1), axis=1)

Traceback (most recent call last):
  File "c:\Users\phamg\.vscode\extensions\ms-python.python-2024.20.0-win32-x64\python_files\python_server.py", line 130, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
  File "C:\Users\phamg\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\frame.py", line 10374, in apply
    return op.apply().__finalize__(self, method="apply")
  File "C:\Users\phamg\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\apply.py", line 916, in apply
    return self.apply_standard()
  File "C:\Users\phamg\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\apply.py", line 1063, in apply_standard
    results, res_index = self.apply_series_generator()
  File "C:\Users\phamg\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\apply.py", line 1081, in apply_series_generator
    results[i] = self.func(v, *self.args, **self.kwargs)
  File "<stri

In [ ]:
print(processed_df)

      Year  Month  Day  ...  StnPress_max  StnPress_min  HMDX
0     2015      1    1  ...        100.45         99.53   0.0
1     2015      1    2  ...        101.31         99.73   0.0
2     2015      1    3  ...        101.62         99.98   0.0
3     2015      1    4  ...         99.87         98.55   1.2
4     2015      1    5  ...        101.61         99.43   0.0
...    ...    ...  ...  ...           ...           ...   ...
3434  2024      5   27  ...         99.51         98.42  22.4
3435  2024      5   28  ...         99.75         98.67  20.0
3436  2024      5   29  ...        100.31         99.78  17.8
3437  2024      5   30  ...        100.83        100.35  13.3
3438  2024      5   31  ...        101.18        100.71  15.2

[3439 rows x 17 columns]


#### Thêm HMDX label

In [ ]:
# Thêm cột label dựa trên giá trị HMDX
def categorize_hmdx(hmdx):
    if hmdx < 30:
        return "Safe"
    else:
        return "Unsafe"

In [ ]:
processed_df['HMDX_label'] = processed_df['HMDX'].apply(categorize_hmdx)
print(processed_df)

      Year  Month  Day  Temp_mean  ...  StnPress_max  StnPress_min  HMDX  HMDX_label
0     2015      1    1     -5.092  ...        100.45         99.53   0.0        Safe
1     2015      1    2     -1.729  ...        101.31         99.73   0.0        Safe
2     2015      1    3     -1.417  ...        101.62         99.98   0.0        Safe
3     2015      1    4      2.854  ...         99.87         98.55   1.2        Safe
4     2015      1    5     -7.196  ...        101.61         99.43   0.0        Safe
...    ...    ...  ...        ...  ...           ...           ...   ...         ...
3434  2024      5   27     18.650  ...         99.51         98.42  22.4        Safe
3435  2024      5   28     17.042  ...         99.75         98.67  20.0        Safe
3436  2024      5   29     16.838  ...        100.31         99.78  17.8        Safe
3437  2024      5   30     14.888  ...        100.83        100.35  13.3        Safe
3438  2024      5   31     16.258  ...        101.18        100.7